In [1]:
import json
import pandas as pd 
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

import anndata
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import numpy as np
import pandas as pd
import scipy.stats
import scanpy as sc
import batchglm.api as glm
import diffxpy.api as de

print("batchglm version "+glm.__version__)
print("diffpy version "+de.__version__)

2023-07-19 09:47:19.674059: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-19 09:47:19.675132: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-19 09:47:19.698825: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-19 09:47:19.699229: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 09:47:20.090874: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

batchglm version v0.7.4
diffpy version v0.7.4


In [3]:
import scipy
from statsmodels.stats.multitest import fdrcorrection

In [6]:
len(flat_list)

180000

In [4]:
labels = pd.read_csv("df_new_temp.csv")
my_file = open("DA_regions.txt", "r")
data = my_file.read()
da_regions = data.split("\n")
my_file.close()
flat_list = []
for item in da_regions:
    try:
        item = item.split(']')[1].strip()  # Remove the part before ']'
        flat_list.extend(item.split())  # Split and extend the flat_list
    except:
        break
        
labels = labels.drop("Unnamed: 0", axis=1)
#labels = labels.set_index("label")
labels["da_regions"] = flat_list
labels["da_regions"] = labels["da_regions"].astype(int)
#labels["condition"] = np.where(labels['label'].str.contains("ALS"), "Case", "Control")
labels["condition"] = np.where(labels['label'].str.contains("ALS"), 1, 0)

labels["sample"] = labels["label"].str[:-12]

ValueError: Length of values (180000) does not match length of index (189595)

In [ ]:
adata = sc.AnnData(X=labels.iloc[:,:-4].to_numpy(), obs=labels[["condition", "da_regions"]])

In [ ]:
adata

In [ ]:
test = de.test.pairwise(
    data=adata,
    grouping="da_regions",
    test="z-test",
    lazy=False,
    noise_model="nb"
)

In [ ]:
labels["da_regions"]

In [ ]:
labels

In [ ]:
crash

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
labels["da_regions"]

In [ ]:
#plt.clf()
sns.displot(labels, x="da_regions", hue="condition", kind="hist", multiple="stack", bins=len(np.unique(labels["da_regions"].values)))
plt.yscale("log")

In [ ]:
#plt.clf()
sns.displot(labels, x="da_regions", hue="label", kind="hist", multiple="stack", bins=len(np.unique(labels["da_regions"].values)))
plt.yscale("log")

In [ ]:
#plt.clf()
#labels = labels[labels["da_regions"] != 0]
sns.displot(labels[labels["condition"] == "Case"] , x="da_regions", hue="sample", kind="hist", multiple="stack", bins=len(np.unique(labels["da_regions"].values)))
#plt.yscale("log")
plt.xticks(range(len(np.unique(labels["da_regions"].values))))
plt.show()

In [ ]:
#plt.clf()
#labels = labels[labels["da_regions"] != 0]
sns.displot(labels[labels["condition"] == "Control"] , x="da_regions", hue="sample", kind="hist", multiple="stack", bins=len(np.unique(labels["da_regions"].values)))
#plt.yscale("log")
plt.xticks(range(len(np.unique(labels["da_regions"].values))))
plt.show()

In [ ]:
plt.clf()
sns.displot(labels, x="da_regions", y='CD11b-PE', bins=12)

In [ ]:
#plt.clf()
#labels = labels[labels["da_regions"] != 0]
sns.displot(labels[labels["da_regions"] != 0] , x="da_regions", hue="sample", kind="hist", multiple="stack", bins=len(np.unique(labels["da_regions"].values)))
#plt.yscale("log")
plt.xticks(range(len(np.unique(labels["da_regions"].values))))
plt.show()

In [ ]:
regions = np.unique(labels["da_regions"].values)

genes = np.array(labels.columns[:-4])
de = np.ndarray((len(regions), len(genes))) 
    
for i, region in enumerate(regions):
    pvals = {}

    for gene in genes:
        x_cells = labels[labels["da_regions"] == region]
        y_cells = labels[labels["da_regions"] != region]
        x = x_cells[gene].values        
        y = y_cells[gene].values
        pvals[gene] = scipy.stats.mannwhitneyu(x,y).pvalue
    rejected = fdrcorrection(list(pvals.values()))
    degs_for_region = genes[np.where(rejected[0])]
    de[i] = rejected[0]

In [ ]:
pvals

In [ ]:
res = pd.DataFrame(de, columns=genes, index=regions, dtype=bool)

In [ ]:
res

In [ ]:
%pip install pydeseq2